<a href="https://colab.research.google.com/github/VejayPersaud/Microbiome_Data/blob/main/finalproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# microbiome disease prediction with transformers

The final course project is designed to be an open-ended and 'creative' analysis of microbial community data, with the goal of training a model to predict autoimmune disease risk from microbiome data.

The data intake and packaging pipeline is coded for you in the following code cell; it includes the addition of 'location' information in the sequence data, so it can be directly used in transformer models.

We will be using the same microbial community data that was used in week 12, and you will be required to implement a transformer model similar to the one we implemented in week 13, so you may want to review the material from those weeks before getting started.

But this final project is a chance for you to 'play around' with *all* the various tools and techniques we've looked at throughout this course, so take a little time to reflect on some of the problems and approaches we've worked on; these are all 'tools' you can use for data analysis.

The grade for the final project will be based on a structured 'write up' of what you do in this jupyter notebook. This is a chance for you to explore the concepts and techniques we've introduced in this course in more detail, and get points for doing so. So think creatively!

I'd recommend exploring ideas and analysis options by conducting a series of 'experiments' or data analyses. In each 'experiment', you make a *change* to *how* you are analyzing the data. You might use a different optimizer, or add another layer to your network. *Which* changes you make should be guided by *questions* or *problems* you encounter. For example, your model's accuracy might be low, and you might try to *improve* accuracy by adding a new layer to your network. For another example, if you observe overfitting in your model, you might try adding a regularizer to reduce overfitting. You then observe the *effect* of each change you make on the data analysis, so you can evaluate what the change does. For example, does adding another layer to the network improve accuracy? Does it also increase overfitting?

For each 'experiment' you conduct in this notebook, you'll want to keep track of the following information:

1. What *question* is the experiment or analysis trying to answer? What do you want to *know* by doing the analysis or experiment?
2. What exactly did you *do*? What changes did you make that you want to assess?
3. What happened? What information or 'data' was collected during your analysis or experiment?
4. What did the 'results' of your experiment or analysis *tell you* that is *relevant* to answering the question that prompted you to perform this experiment or analysis?

Every time you do an analysis or change something about the way you are modeling or analyzing the data, keep track of your answers to questions 1-4 above, and then you can use that information for your final project write up.

In [4]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')


# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

print(train_data, valid_data)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 2), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))> <_BatchDataset element_spec=(TensorSpec(shape=(None, 256, 2), dtype=tf.float64, name=None), TensorSpec(shape=(None,), dtype=tf.float64, name=None))>


From the Dataset information printed to the screen, you can see that the shape of your input data is (256,2). You might also want to note that you have a binary classification problem.

The data intake and packaging pipeline is reproduced in the following code cell. Your job is to implement the code that builds, compiles and fits a transformer model to these data, including model evaluation.

I'd recommend starting with a very 'simple' transformer model, to get a baseline accuracy measurement.

You might also want to try changing the random seed used to create the train-validate data split, so you can evaluate your model's accuracy using *different* train-validate splits. For example, if you *average* your accuracy measurements over *many different* train-validate splits, you'll probably get a better estimate of your model's *expected* accuracy (this is called "cross-validation", and it is commonly used in machine learning).

You might also want to try changing the number of epochs used for training, to see how short you can make your training runs, while still training your model 'to convergence' (ie, stable loss values). You could also try using different optimizers, to see if that impacts how long you need to train your model.

After you get a good 'feeling' for the behaviour of your 'baseline' transformer model for this problem, try altering the model in various ways, and observe the impact on training time and accuracy. For example, you will probably want to try changing the dimensionality of the internal data representation in your model. You will probably also want to try changing the number of attention 'heads' in your multi-head attention layer. You could also try removing the normalization layers in your model, to see if layer normalization is needed.

You may also want to try replicating the *entire* transformer block multiple times; try creating a model with 2 or 4 sequential transformer blocks, one after the other, before the decision layers. At what point does your model become 'too complex' and start 'overfitting' the training data? Does adding dropout or regularization help control overfitting in your transformer model?

Make sure you keep track of each change you try, and the effect of the various changes. Remember that you can add new text and code cells to this jupyter notebook, so you could replicate the code cell below to try out different things, and add text cells to keep notes of what you are trying out, why and the results of each 'experiment'.

Finally, you might want to consider comparing the transformer model to alternative sequence models for this problem. How does the transformer model's accuracy compare to the accuracy of a Conv1D or LSTM model? Which models are faster to train? Which models have more trainable parameters? Are any models inherently more/less susceptible to overfitting? You might want to also try testing variants of Conv1D and LSTM that *include* the linear projection layer that projects your data into a high-dimensional representational space, so the results are more 'comparable' between the transformer and the other sequence models.

Baseline Model

In [7]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

## build model
repdim = 4  # internal representational dimensionality

# input and linear projection
inlayer = tf.keras.Input(shape=(256,2))
proj    = tf.keras.layers.Dense(units=repdim)(inlayer)

# multi‑head attention block
mha1 = tf.keras.layers.MultiHeadAttention(num_heads=1, key_dim=repdim)(proj, proj, proj)
res1 = tf.keras.layers.Add()([proj, mha1])
nrm1 = tf.keras.layers.LayerNormalization()(res1)

# feed‑forward block
ffa1 = tf.keras.layers.Dense(units=repdim, activation=tf.keras.activations.relu)(nrm1)
ffb1 = tf.keras.layers.Dense(units=repdim)(ffa1)
res2 = tf.keras.layers.Add()([nrm1, ffb1])
nrm2 = tf.keras.layers.LayerNormalization()(res2)

# classification block
flt     = tf.keras.layers.Flatten()(nrm2)
outlayer = tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(flt)

model = tf.keras.Model(inputs=inlayer, outputs=outlayer)

model.summary()

## compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

## fit model
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data)

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 256, 2)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 256, 4)    │         12 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 256, 4)    │         80 │ dense_8[0][0],    │
│ (MultiHeadAttentio… │                   │            │ dense_8[0][0],    │
│                     │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 256, 4)    │          0 │ dense_8[0][0],    │
│                     │                   │            │ multi_head_atten… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256, 4)    │          8 │ add_4[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 256, 4)    │         20 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 256, 4)    │         20 │ dense_9[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_5 (Add)         │ (None, 256, 4)    │          0 │ layer_normalizat… │
│                     │                   │            │ dense_10[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 256, 4)    │          8 │ add_5[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 1024)      │          0 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │      1,025 │ flatten_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,173 (4.58 KB)

 Trainable params: 1,173 (4.58 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 23s 47ms/step - accuracy: 0.5451 - loss: 0.6931 - val_accuracy: 0.7177 - val_loss: 0.5764
Epoch 2/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 43ms/step - accuracy: 0.7374 - loss: 0.5297 - val_accuracy: 0.7697 - val_loss: 0.4733
Epoch 3/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - accuracy: 0.7948 - loss: 0.4448 - val_accuracy: 0.7898 - val_loss: 0.4413
Epoch 4/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 18s 43ms/step - accuracy: 0.8060 - loss: 0.4183 - val_accuracy: 0.8018 - val_loss: 0.4189
Epoch 5/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 19s 46ms/step - accuracy: 0.8172 - loss: 0.4010 - val_accuracy: 0.8088 - val_loss: 0.4062
Epoch 6/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 20s 44ms/step - accuracy: 0.8210 - loss: 0.3897 - val_accuracy: 0.8155 - val_loss: 0.3982
Epoch 7/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 18s 44ms/step - accuracy: 0.8261 - loss: 0.3817 - val_accuracy: 0.8174 - val_loss: 0.3936
Epoch 8/20
409/409 ━━━━━━━━━━━━━━━━━━━━ 18s 45ms/step - accuracy: 0.8280 - loss: 0.3757 - 

Experiment 1

In [ ]:
import pandas
import numpy as np
import tensorflow as tf

# download data
dataframe = pandas.read_csv('https://raw.githubusercontent.com/bryankolaczkowski/ALS3200C/main/mbiome.data.csv')

# create train-validate split
train_dataframe = dataframe.sample(frac=0.8, random_state=827847)
valid_dataframe = dataframe.drop(train_dataframe.index)

# extract explanatory variables
dta_ids = [ x for x in dataframe.columns if x.find('DTA') == 0 ]
train_x = train_dataframe[dta_ids].to_numpy()
valid_x = valid_dataframe[dta_ids].to_numpy()

# add 'location' to sequence data
loc = np.linspace(start=-2.5, stop=+2.5, num=train_x.shape[1])
train_x = np.stack([ train_x, np.array([loc]*train_x.shape[0]) ], axis=-1)
valid_x = np.stack([ valid_x, np.array([loc]*valid_x.shape[0]) ], axis=-1)

# extract labels
train_y = train_dataframe['LBL0'].to_numpy()
valid_y = valid_dataframe['LBL0'].to_numpy()

# package data into tensorflow dataset
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y)).batch(32)
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(32)

## build model
repdim = 4  # internal representational dimensionality

# input and linear projection
inlayer = tf.keras.Input(shape=(256,2))
proj    = tf.keras.layers.Dense(units=repdim)(inlayer)

# multi‑head attention block
mha1 = tf.keras.layers.MultiHeadAttention(num_heads=1, key_dim=repdim)(proj, proj, proj)
res1 = tf.keras.layers.Add()([proj, mha1])
nrm1 = tf.keras.layers.LayerNormalization()(res1)

# feed‑forward block
ffa1 = tf.keras.layers.Dense(units=repdim, activation=tf.keras.activations.relu)(nrm1)
ffb1 = tf.keras.layers.Dense(units=repdim)(ffa1)
res2 = tf.keras.layers.Add()([nrm1, ffb1])
nrm2 = tf.keras.layers.LayerNormalization()(res2)

# classification block
flt     = tf.keras.layers.Flatten()(nrm2)
outlayer = tf.keras.layers.Dense(units=1, activation=tf.keras.activations.sigmoid)(flt)

model = tf.keras.Model(inputs=inlayer, outputs=outlayer)

model.summary()

## compile model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

## fit model
history = model.fit(train_data,
                    epochs=20,
                    validation_data=valid_data)